In [ ]:
%matplotlib qt
%gui qt5
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import os
import os.path as op
import matplotlib.pyplot as plt
import mne
import csv
import batch_average as ba
from sklearn.utils import shuffle
import sklearn
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV

### Importing preprocessed data - epochs

In [ ]:
overwrite = True
eeg_path = r'D:\Master_thesis\Pre-processed_data'
data_path_out = r'D:\Master_thesis\Scores\Binary classification\Stereo_ERP_windows'

### Windows ERP- Stereo vs Mono

In [ ]:
emotion_combinations = [["mono/happy", "mono/neutral"],["mono/happy", "mono/angry"], ["mono/happy", "mono/surprised"], ["mono/neutral", "mono/angry"], ["mono/neutral", "mono/surprised"], 
                       ["mono/angry", "mono/surprised"]],[["stereo/happy", "stereo/neutral"], ["stereo/happy", "stereo/angry"], ["stereo/happy", "stereo/surprised"], 
                       ["stereo/neutral", "stereo/angry"], ["stereo/neutral", "stereo/surprised"], ["stereo/angry", "stereo/surprised"]]
                        
                        
emo_comb_num = [1, 2, 3, 4, 5, 6]
scores_avg_all = []
scores_all  = []


l1_ratio = [0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
C = [0.1, 0.2, 0.3, 0.5, 0.6, 0.8, 0.9, 1, 1.1, 1.2, 1.5, 1.7, 2, 3, 4]
scoring = 'accuracy' 

viewing_condition = [0, 1]


n_split = np.arange(1, 6 , dtype = int)

# n_split = [1]


windows_time = ["P1", 0.08, 0.12], ["N170", 0.14, 0.20] ,["EPN", 0.15, 0.35], ["LPC", 0.40, 0.60]
win_comb_num = [1, 2, 3, 4]
# win_comb_num = [1]

subjects = ['VR2FEM_S07', 'VR2FEM_S08', 'VR2FEM_S10', 'VR2FEM_S11', 'VR2FEM_S12', 'VR2FEM_S14', 'VR2FEM_S15', 'VR2FEM_S17', 'VR2FEM_S18', 'VR2FEM_S21']

for subject in subjects:
    data_path = os.path.join(eeg_path, subject)
            #reading preprocessed data -epochs
    epochs_train_list = []
    epochs_test_list = []
    for items in n_split:
        epochs_train_name = os.path.join(f"{data_path}\{subject}-preprocessed_train_{items}-epo.fif")
        epochs_test_name = os.path.join(f"{data_path}\{subject}-preprocessed_test_{items}-epo.fif")
        epochs_train = mne.read_epochs(epochs_train_name, preload=True)
        epochs_test = mne.read_epochs(epochs_test_name, preload=True)
        epochs_train_list.append(epochs_train)
        epochs_test_list.append(epochs_test)

    for view in viewing_condition:
        print(f"Viewing Condition: {view}")
        for com_num in emo_comb_num:
            print(f"Number of emo combination: {com_num}")
            for windows in win_comb_num:
                wins = windows-1
                print(f"Window split: {windows_time[wins][0]}")
                scores_avg_all = []
                for l1 in l1_ratio:
                    for cs in C:
                        print("Running reg. ratio for: ",l1)
                        print("Running reg. strength for: ",cs)   
                        clf = make_pipeline(
                        StandardScaler(),
                        LogisticRegression(max_iter = 4000, solver='saga',penalty='elasticnet', l1_ratio=l1, C=cs))

                        scores_all  = []

                        for split in n_split:
                            n = split-1 
                            print(f"Number of split: {split}")
                            scores_all_scaler  = []
                            epochs_train = epochs_train_list[n][emotion_combinations[view][com_num-1][0], emotion_combinations[view][com_num-1][1]].copy().crop(windows_time[wins][1], windows_time[wins][2])
                            events_train = epochs_train_list[n][emotion_combinations[view][com_num-1][0], emotion_combinations[view][com_num-1][1]].events[:,2]
                            x_train = np.mean(epochs_train, axis = 2)
                            y_train = [int(str(yy)[-1]) for yy in events_train]
    #                             scaler.fit(x_train)
    #                             x_train_scaler = scaler.transform(x_train)

                            epochs_test = epochs_test_list[n][emotion_combinations[view][com_num-1][0], emotion_combinations[view][com_num-1][1]].copy().crop(windows_time[wins][1], windows_time[wins][2])
                            events_test = epochs_test_list[n][emotion_combinations[view][com_num-1][0],emotion_combinations[view][com_num-1][1]].events[:,2]
                            x_test = np.mean(epochs_test, axis = 2)
                            y_test = [int(str(yy)[-1]) for yy in events_test]
    #                             scaler.fit(x_test)
    #                             x_test_scaler = scaler.transform(x_test)


                            clf.fit(x_train, y_train)
                            clf.predict(x_test)
                            score_scaler = clf.score(x_test, y_test)
                        #     scores_all_scaler.append(score_scaler)
                            scores_all.append(score_scaler)

                        scores_CV= np.vstack(scores_all)
#                         print(scores_CV)
                        scores_avg = np.mean(scores_CV, axis=0)
#                         print(scores_avg)
                        scores_avg_all.append(scores_avg)

                scores_avg_best = max(scores_avg_all)

    #                 print("scores_avg_scaler_best", scores_avg_scaler_best.item())
                print(f"Average score: {scores_avg_best}")
                dir_save= os.path.join(data_path_out, subject)
                if not op.exists(dir_save):
                            os.makedirs(dir_save)
                score_file_name = f"{dir_save}\{subject}_{emotion_combinations[view][com_num-1][0].replace('/','__')} vs {emotion_combinations[view][com_num-1][1].replace('/','__')}_{windows_time[wins][0]}.csv"
                print(score_file_name)
                with open(score_file_name, "w") as file: 
                    writer = csv.writer(file)
                    writer.writerow([scores_avg_best.item()])
